In [ ]:
pip install langchain langchain-openai faiss-cpu openai
pip install -U langchain-community


In [ ]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_APIKEY")

In [ ]:
# Load documents & create vector store (RAG setup)
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS # Import FAISS from langchain_community
from langchain.text_splitter import CharacterTextSplitter
from langchain.docstore.document import Document

In [ ]:
# Example HR policy text
hr_policy = """
Employees are entitled to 20 paid leave days per year.
Unused leaves cannot be carried forward.
Sick leave requires a doctor’s certificate if more than 3 days.
"""

# Split into chunks
splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
docs = splitter.split_documents([Document(page_content=hr_policy)])

In [ ]:
# Create embeddings + vector store
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(docs, embeddings)

# Create retriever for RAG
retriever = vectorstore.as_retriever()

In [ ]:
from langchain.agents import tool

@tool
def send_email(to: str, subject: str, body: str):
    """Send an email to the given address with subject and body."""
    print(f"📧 Email sent to {to}\nSubject: {subject}\nBody: {body}")
    return "Email sent successfully."


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, AgentType

# LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# RAG Chain (for grounding answers)
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever
)

# Tools: RAG + Email
tools = [
    rag_chain.as_tool(name="hr_policy_lookup", description="Look up HR policy details"),
    send_email
]

# Create AI Agent
# Use AgentType.OPENAI_FUNCTIONS which supports multi-input tools.
# Note: LangGraph's create_react_agent is the recommended approach for new implementations.
agent = initialize_agent(
    tools, llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)

In [ ]:
# User query
task = "Summarize the HR leave policy and email it to deeparajasabeson@gmail.com"

response = agent.run(task)
print("🤖 Agent Response:", response)


/tmp/ipython-input-3712184761.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent.run(task)




> Entering new AgentExecutor chain...

Invoking: `hr_policy_lookup` with `{'query': 'leave policy'}`


{'query': 'leave policy', 'result': "Employees are entitled to 20 paid leave days per year. Unused leave days cannot be carried forward to the next year. If an employee takes sick leave for more than 3 days, a doctor's certificate is required."}
Invoking: `send_email` with `{'to': 'deeparajasabeson@gmail.com', 'subject': 'Summary of HR Leave Policy', 'body': "Dear Deepa,\n\nHere is a summary of the HR leave policy:\n\n- Employees are entitled to 20 paid leave days per year.\n- Unused leave days cannot be carried forward to the next year.\n- If an employee takes sick leave for more than 3 days, a doctor's certificate is required.\n\nBest regards,\nHR Team"}`


📧 Email sent to deeparajasabeson@gmail.com
Subject: Summary of HR Leave Policy
Body: Dear Deepa,

Here is a summary of the HR leave policy:

- Employees are entitled to 20 paid leave days per year.
- Unused leave days cannot be